In [2]:
# inject high intensity signals to verify that we can recover them after they go through SM-Emp 
# in the shifted spectra and the residuals.
# Then should check that they are recovered by the search algorithm, of course.

import numpy as np
import os
import matplotlib.pyplot as plt
from astropy.io import fits
import pandas as pd
import math
from scipy.interpolate import interp1d
from mpmath import mp
mp.dps=100
exp_array = np.frompyfunc(mp.exp, 1, 1)

In [3]:
# make a directory for the injected spectra -- only run this once
#if not os.path.isdir('/mnt_home/azuckerman/BL_APF_DAP/APF_spectra/injected_spectra'):
#    os.mkdir('/mnt_home/azuckerman/BL_APF_DAP/APF_spectra/injected_spectra')

In [4]:
# function to insert simulated gaussians
def insert_gaussian(spectrum, gaussian_params, midpoint, numpoints):
    height = gaussian_params[0]
    position = gaussian_params[1] #position within segment, not index in spectrum
    FWHM = gaussian_params[2]
    offset = gaussian_params[3]
    x = np.linspace(0,numpoints-1,numpoints) # numpoints must be odd
    gauss = gaussian(x,height,position,FWHM/(2*np.sqrt(2*np.log(2))),offset)
    new_spect = spectrum
    new_spect[midpoint - math.floor(numpoints/2):midpoint + math.floor(numpoints/2)] = spectrum[midpoint - math.floor(numpoints/2):midpoint + math.floor(numpoints/2)] + gauss
    return new_spect
    
def gaussian(x,a,b,c,d): # a = height, b = position of peak, c = width, x = numpy array of x values
    f = a*exp_array((-(x-b)**2)/(2*c)**2) + d
    return f 

In [5]:
test_stars = ['GJ1002', # -> cool (T ~ 3100K)
              'HIP115562', # -> relatively cool (T ~ 3700K)
              'GJ144', # -> medium temperature (T ~ 5000K)
              'HIP13834'] # -> hot (T ~ 6300K)

star_list = os.listdir('APF_spectra/all_apf_spectra_highest_SNR')[:100]

heights = [1.5, 2, 3] # times the 98th percentile of order 40


#if not os.path.isdir('APF_spectra/injected_spectra_' + str(height)):
#   os.mkdir('APF_spectra/injected_spectra_' + str(height))
apf_wave = fits.open('apf_wav.fits')
j = 0
injected_peak_photons = np.zeros(len(star_list)*3) # *3 becuase three different inejctions in each star
median_photons_100_pix = np.zeros(len(star_list)*3)
injection_wl = np.zeros(len(star_list)*3)*np.nan
stars = []
if not os.path.isdir('APF_spectra/injected_spectra'):
    os.mkdir('APF_spectra/injected_spectra')
    
for star_name in star_list:
    star = star_name.split('_')[0]
    for m in range(3): stars += [star] # want three entries for this star: one for each injection
    dir_path = 'APF_spectra/all_apf_spectra_highest_SNR/' + star + '_spectra'
    i = 0
    print('Star: ' + star)
    for file in os.listdir(dir_path):
        hdu = fits.open(dir_path + '/' + file)# , mode='update')
        spectrum = np.copy(hdu[0].data)
        color = 'C' + str(i)  
        n = 0
        orders = [39, 40, 41]
        y = hdu[0].data
        for height in heights:
            order = orders[n] # inject the 1.5 hieght into order 39, 2 into order 40, etc
            y_order = y[order]
            h = height - 1 # becuase in the above implementation we add to the baseline, so ie. h = 1 injects a guassian whose peak is at twice the pixels value of the 98th percentile
            print('    file ' + file + ': injecting signal of W = 5, H = ' + str(height) + 
                  '*(98th pct level of order) = ' + str(np.round(height * np.percentile(y_order,98),3)) + ' into order 40.')  
            median_100 = np.median(y_order[int(len(y_order)/2 - 50): int(len(y_order)/2 + 50)])
            y_new = insert_gaussian(y_order, [height*np.percentile(y_order,98), 49, 20, 0], int(len(y_order)/2), 100)
            injection_center_wl = apf_wave[0].data[order][int(len(y_order)/2)]
            print('                     : injecting at center point ' + str(injection_center_wl) + ' A')
            #y[int(len(y)/2)] = 10 * np.percentile(y,99)
            #x = apf_wave[0].data[order]
            #plt.plot(x, y[:-1], '.', c = color)
            #plt.xlim([5553,5557])
            injected_peak_photons[3*j + n] = injected_peak_photons[3*j + n] + height * np.percentile(y_order,98) # sum over each file
            median_photons_100_pix[3*j + n] = median_photons_100_pix[3*j + n] + median_100 # sum over each file
            injection_wl[3*j + n] = injection_center_wl # same for each file (and each star)
            spectrum[order,:] = y_new
            n += 1
        #plt.figure(star + ', ' + file)
        #for order in np.arange(38,43,1):
        #    plt.plot(apf_wave[0].data[order], spectrum[order,:-1])
        #    plt.title(star + ', ' + file)
        i += 1

        new_header = hdu[0].header # get header
        new_header.set('Injected', 'YES','Spectrum contains injected gaussian for testing.')
        data_hdu = fits.PrimaryHDU(spectrum, new_header) 
        hdu_new = fits.HDUList([data_hdu])
        out_path = 'APF_spectra/injected_spectra/'
        if not os.path.isdir(out_path + star + '_spectra/'):
            os.mkdir(out_path + star + '_spectra/')
        #hdu_new.writeto(out_path + star + '_spectra/' + file + '_injected' + '.fits')
        hdu.close()
    save_data = list(zip(stars, injection_wl, injected_peak_photons, median_photons_100_pix))
    df = pd.DataFrame(save_data,
              columns = ['Star_name', 'Injection_wavelength', 'Injected_peak_photons', 'Local_median_photons'])
    df.to_csv('Injection_key.csv', index = False)
    j += 1

Star: GJ1002
    file rbhr.214.fits: injecting signal of W = 5, H = 1.5*(98th pct level of order) = 151.303 into order 40.
                     : injecting at center point 5490.07413621209 A
    file rbhr.214.fits: injecting signal of W = 5, H = 2*(98th pct level of order) = 153.809 into order 40.
                     : injecting at center point 5555.36527738973 A
    file rbhr.214.fits: injecting signal of W = 5, H = 3*(98th pct level of order) = 298.692 into order 40.
                     : injecting at center point 5622.229264717167 A
    file rbhr.215.fits: injecting signal of W = 5, H = 1.5*(98th pct level of order) = 155.19 into order 40.
                     : injecting at center point 5490.07413621209 A
    file rbhr.215.fits: injecting signal of W = 5, H = 2*(98th pct level of order) = 163.734 into order 40.
                     : injecting at center point 5555.36527738973 A
    file rbhr.215.fits: injecting signal of W = 5, H = 3*(98th pct level of order) = 301.976 into order

In [22]:
test_stars[0]

'GJ1002'

In [69]:
hdu = fits.open(dir_path + '/' + file)# , mode='update')
np.shape(hdu[0].data)

(79, 4608)

In [74]:
hdu.info()

Filename: APF_spectra/all_apf_spectra_highest_SNR/HIP13834_spectra/rceq.102.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     237   (4608, 79)   float32   
